In [31]:
# Importing all the libraries needed
import numpy as np
import pandas as pd
import scipy 
import matplotlib as mplt
import os
from pydub import AudioSegment
from python_speech_features import mfcc, logfbank, ssc
import scipy.io.wavfile as wav
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [5]:
# Some variable and path initializations
os.chdir("/home/aayush/Apogee_project/apogee_amgc/genres1")
ori_folder_flag = 1
song_no = 0
df = pd.DataFrame(columns=['mfcc_feat','fbank_feat','ssc','classification'],index=range(0,1000))

In [6]:
# Getting all the genres
l = []
for dirpaths,dirnames,filenames in os.walk(os.getcwd()):
    l.append(dirnames)

In [7]:
# Extracting Features of the songs

for i in l[0]:
    #for genre in genres1
    for x in range(100):
        # for song in batch of songs
        
        # Extracting the features
        (rate,sig) = wav.read(i+'/'+i+'.000'+"%02d"%x+'.wav')
        mfcc_feat = mfcc(sig,rate,nfft=551)
        fbank_feat = logfbank(sig,rate,nfft=551)
        sig_temp = np.reshape(sig, (sig.shape[0],1))
        ssc_var = ssc(sig_temp,rate,nfft=551)
        
        #Adding features to the pandas dataframe
        df.iloc[song_no][0] = mfcc_feat[0:2992,:]
        df.iloc[song_no][1] = fbank_feat[0:2992,:]
        df.iloc[song_no][2] = ssc_var[0:2992,:]
        df.iloc[song_no][3] = i
        
        # Incrementing the song number
        song_no+=1
        
    

In [46]:
train, test = train_test_split(df, test_size=0.2)

train_values = train.loc[:,features].values


# t = StandardScaler().fit_transform(train_values[0])
print(train_values[0,0].shape)
print(train_values[0,1].shape)
print(train_values[0,2].shape)
t = np.concatenate( (train_values[0,0],train_values[0,1],train_values[0,2]), axis = 1)
print(t.shape)

(800, 3)
(2992, 13)
(2992, 26)
(2992, 26)
(2992, 65)


In [37]:
features = ['mfcc_feat','fbank_feat','ssc']

#Separating out the features, fitting and tranforming the train and test data
train_values = train.loc[:,features].values # returns an array from the dataframe

for i in range(3):
    t = StandardScaler().fit_transform(train_values[0])

train_normalized = StandardScaler().fit_transform(train_values)

# Transforming with the same scale for test values
test_values = train.loc[:,features].values
test_normalized = StandardScaler().transform(test_values)

#Separating out the target
train_classification = train.iloc[:,['classification']].values
test_classification = test.iloc[:,['classification']].values

# Number of components is 8
pca = PCA( n_components = 8) 

principal_components_train = pca.fit_transform(train_normalized)
principal_components_test = pca.fit_transform(test_normalized)

pca_train_temp = pd.Dataframe(data = principal_components_train, columns = ['pca1','pca2','pca3','pca4','pca5','pca6','pca7','pca8']) 
pca_test_temp = pd.Dataframe(data = principal_components_test,columns = ['pca1','pca2','pca3','pca4','pca5','pca6','pca7','pca8'])

pca_train_df = pd.concat( [pca_train_temp,train_classification], axis = 1 )
pca_test_df = pd.concat( [pca_test_temp,test_classification], axis = 1 )

print(pca_train_df)
print(pca_test_df)


AttributeError: 'numpy.ndarray' object has no attribute 'iloc'